In [37]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import cv2
import h5py
from tqdm import tqdm

In [40]:
# fetch data
with h5py.File("./tests/data/mujoco_aligned_tracking.h5", 'r') as f:
    mujoco_data = f['tracks'][:]

with h5py.File("./tests/data/mujoco_output_kp_data.h5", 'r') as f:

    output_data = f['tracks'][:]
    output_data = np.expand_dims(output_data, axis=1)

In [101]:
def single_plotter(points3d, output_video, start_frame=0, num_frames=None):
    if not output_video.endsWith('.mp4'):
        print("filename does not end with '.mp4'")
        return

    # Define start frame and number of frames to process
    if not num_frames:
        num_frames = points3d.shape[0]
    end_frame = start_frame + num_frames

    # Define skeleton connections (edges)
    connections = [(0, 1), (1, 2)]  # Adjust based on data structure
    nodes = ["shoulder", "elbow", "wrist"]  # Node names for visualization

    # Compute global axis limits for transformed data
    x_min = points3d[..., 0].min()
    x_max = points3d[..., 0].max()
    y_min = points3d[..., 1].min()
    y_max = points3d[..., 1].max()
    z_min = points3d[..., 2].min()
    z_max = points3d[..., 2].max()

    # Prepare video writing
    fps = 40  # Frames per second

    # Initialize video writer
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_zlim(z_min, z_max)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')

    # Create canvas
    canvas = FigureCanvas(fig)

    fourcc = cv2.VideoWriter_fourcc(*'avc1')
    video_writer = cv2.VideoWriter(output_video, fourcc, fps, (canvas.get_width_height()[::-1]))

    for frame_idx in tqdm(range(start_frame, end_frame), desc="Generating frames"):
        ax.clear()

        # Plot new coordinate axes
        # ax.quiver(0, 0, 0, 1, 0, 0, color='blue', length=5, arrow_length_ratio=0.1)
        # ax.quiver(0, 0, 0, 0, 1, 0, color='green', length=5, arrow_length_ratio=0.1)
        # ax.quiver(0, 0, 0, 0, 0, 1, color='red', length=5, arrow_length_ratio=0.1)

        # Get transformed coordinates for the selected frame and track
        coords = points3d[frame_idx, 0]

        # Plot nodes
        for i in range(3):
            ax.scatter(coords[i, 0], coords[i, 1], coords[i, 2], c='b')
            ax.text(coords[i, 0], coords[i, 1], coords[i, 2], f"{nodes[i]}", fontsize=10, color="blue")

        # Plot skeleton connections
        for start, end in connections:
            ax.plot(
                [coords[start, 0], coords[end, 0]],
                [coords[start, 1], coords[end, 1]],
                [coords[start, 2], coords[end, 2]],
                'k-', linewidth=2
            )

        # Maintain consistent axis limits
        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)
        ax.set_zlim(z_min, z_max)

        # Render the frame
        canvas.draw()
        frame = np.frombuffer(canvas.buffer_rgba(), dtype=np.uint8)
        frame = frame.reshape(canvas.get_width_height()[::-1] + (4,))
        video_writer.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

    plt.close(fig)
    video_writer.release()
    print(f"Video saved to {output_video}")

In [99]:
def double_plotter(points1, points2, output_video, start_frame=0, num_frames=None):
    '''
    points1 is mujoco input
    points2 is mujoco output
    '''

    if not output_video.endswith('.mp4'):
        print("filename does not end with '.mp4'")
        return

    # Define start frame and number of frames to process
    if not num_frames:
        num_frames = points1.shape[0]
    end_frame = start_frame + num_frames

    assert points1.shape == points2.shape

    # Define skeleton connections (edges)
    connections = [(0, 1), (1, 2)]  # Adjust based on data structure
    nodes = ["shoulder", "elbow", "wrist"]  # Node names for visualization

    # Compute global axis limits for transformed data
    x_min = np.array(points1[..., 0].min(), points2[..., 0].min()).min()
    x_max = np.array(points1[..., 0].max(), points2[..., 0].max()).max()
    y_min = np.array(points1[..., 1].min(), points2[..., 1].min()).min()
    y_max = np.array(points1[..., 1].max(), points2[..., 1].max()).max()
    z_min = np.array(points1[..., 2].min(), points2[..., 2].min()).min()
    z_max = np.array(points1[..., 2].max(), points2[..., 2].max()).max()

    # Prepare video writing
    fps = 25  # Frames per second

    # Initialize video writer
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_zlim(z_min, z_max)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')

    # Create canvas
    canvas = FigureCanvas(fig)

    fourcc = cv2.VideoWriter_fourcc(*'avc1')
    video_writer = cv2.VideoWriter(output_video, fourcc, fps, (canvas.get_width_height()[::-1]))

    for frame_idx in tqdm(range(start_frame, end_frame), desc="Generating frames"):
        ax.clear()

        # Plot new coordinate axes
        # ax.quiver(0, 0, 0, 1, 0, 0, color='blue', length=5, arrow_length_ratio=0.1)
        # ax.quiver(0, 0, 0, 0, 1, 0, color='green', length=5, arrow_length_ratio=0.1)
        # ax.quiver(0, 0, 0, 0, 0, 1, color='red', length=5, arrow_length_ratio=0.1)

        # Get transformed coordinates for the selected frame and track
        coords1 = points1[frame_idx, 0]
        coords2 = points2[frame_idx, 0]

        # Plot nodes
        for i in range(3):
            ax.scatter(coords1[i, 0], coords1[i, 1], coords1[i, 2], c='b')
            ax.scatter(coords2[i, 0], coords2[i, 1], coords2[i, 2], c='g')
            ax.text(coords1[i, 0], coords1[i, 1], coords1[i, 2], f"{nodes[i]} in", fontsize=10, color="blue")
            ax.text(coords2[i, 0], coords2[i, 1], coords2[i, 2], f"{nodes[i]} out", fontsize=10, color="green")

        # Plot skeleton connections
        for start, end in connections:
            ax.plot(
                [coords1[start, 0], coords1[end, 0]],
                [coords1[start, 1], coords1[end, 1]],
                [coords1[start, 2], coords1[end, 2]],
                '-',
                c = 'purple',
                linewidth=2
            )
            ax.plot(                       
                [coords2[start, 0], coords2[end, 0]],
                [coords2[start, 1], coords2[end, 1]],
                [coords2[start, 2], coords2[end, 2]],
                '-',
                c='lime',
                linewidth=2
            )
        
        # plot red lines between the two models and their euclidian distances
        for i in range(3):
            ax.plot(
                [coords1[i, 0], coords2[i, 0]],
                [coords1[i, 1], coords2[i, 1]],
                [coords1[i, 2], coords2[i, 2]],
                'r-'
            )
            ax.text(
                np.mean([coords1[i, 0], coords2[i, 0]]),
                np.mean([coords1[i, 1], coords2[i, 1]]),
                np.mean([coords1[i, 2], coords2[i, 2]]),
                f"{np.round(np.linalg.norm(coords1[i] - coords2[i]), 4)}",
                fontsize=10,
                verticalalignment='bottom',
                color='red'
            )


        # Maintain consistent axis limits
        ax.set_title(
                     f"input data humerus len: {np.round(mujoco_in_humerus_len, 4)}\
                     \ninput data radius len: {np.round(mujoco_in_radius_len, 4)}")
        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)
        ax.set_zlim(z_min, z_max)

        # Render the frame
        canvas.draw()
        frame = np.frombuffer(canvas.buffer_rgba(), dtype=np.uint8)
        frame = frame.reshape(canvas.get_width_height()[::-1] + (4,))
        video_writer.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

    plt.close(fig)
    video_writer.release()
    print(f"Video saved to {output_video}")

In [81]:
mujoco_in_humerus_len = np.linalg.norm(mujoco_data[:, 0, 0] - mujoco_data[:, 0, 1], axis=1).mean()
mujoco_in_radius_len = np.linalg.norm(mujoco_data[:, 0, 1] - mujoco_data[:, 0, 2], axis=1).mean()
mujoco_out_humerus_len = np.linalg.norm(output_data[:, 0, 0] - output_data[:, 0, 1], axis=1).mean()
mujoco_out_radius_len = np.linalg.norm(output_data[:, 0, 1] - output_data[:, 0, 2], axis=1).mean()

In [100]:
double_plotter(mujoco_data, output_data, 'mujoco_input_vs_output.mp4', start_frame=0, num_frames=1000)

Generating frames: 100%|██████████| 1000/1000 [01:36<00:00, 10.36it/s]

Video saved to mujoco_input_vs_outout.mp4
